In [ ]:
import pandas as pd
import numpy as np
import sympy as sp
import matlplotlib.pyplot as plt
import openpyxl import loadworkbook

In [ ]:
 codigo = "2200203"
X = sum([int(i) for i in codigo])
Y = X / 3
L1 = 0.05 * X
L2 = L1 * 1.5
L3 = L1 * 0.5
L_T = L1 + L2 + L3
W = X * 0.005 + Y * 0.01
P = X * 0.05
Ax, Ay, MA, x = sp.symbols(“Ax Ay MA x)
Angulo = 45
Angulo_rad = Angulo * np.pi / 180
Px = np.sin(Angulo_rad) * P
Py = Px
L_T

In [ ]:
#HACER ECUACIONES DE EQUILIBRIO
#kN
Equilibrio_y = Ay + Py – W * (L1 + L3) / 2 – W * L2
Ecuacion_y = sp.Eq(Equilibrio_y,0)
Fuerza_Ay = sp.solve(Ecuacion_y)[0]

Fuerza_Ay

In [ ]:
Equilibrio_x = Ax – Px 
Ecuacion_X = sp.Eq(Equilibrio_x, 0)
Fuerza_Ax = sp.solve(Ecuacion_x)[0]

Fuerza_Ax

In [ ]:
Equilibrio_Momentos = MA + Py * L_T – W * L1 / 2 *(2 * L1 / 3) – W * L2 * (L1 + L2 / 2) – W * L3
Ecuacion_Momentos = sp.Eq(Equilibrio_Momentos, 0)
Momento_A = sp.solve(Ecuacion_Momentos)[0]

Momento_A

In [ ]:
#Diagramas de cortante y momento
#Primer Tramo
# 0 < x < L1
Funcion_W = W * x / L1
Cortante_1 = Fuerza_Ay – sp.integrate(Funcion_W, x)
Momento_1 = - Momento_A + sp.integrate(Cortante_1, x)

In [ ]:
Cortante_1

In [ ]:
Momento_1

In [ ]:
#Segundo Tramo
# 0 < x < L2
Funcion_W = W
Cortante_2 = cortante_1.subs(x, L1) – sp.integrate(Funcion_W, x)
Momento_2 = Momento_1.subs(x, L1) + sp.integrate(Cortante_2, x)

In [ ]:
Momento_2

In [ ]:
#Tercer Tramo
# 0 < x < L3
Funcion_W = - W * x / L3 + W
Cortante_3 = Cortante_2.subs(x,L2) – sp.integrate(Funcion_W, x)
Momento_3 = Momento_2.subs(x, L2) + sp.integrate(Cortante_3, x)

In [ ]:
Cortante_3

In [ ]:
Momento_3

In [ ]:
#Graficas
#CORTANTE
x_array = np.arange(0, L_T + 0.005, 0.005)
#Ejecutar
Tramo1 = sp.lambdify(x, Cortante_1)(x_array[x_array < L1])
Tramo2 = sp.lambdify(x, Cortante_2)(x_array[(x_array >= L1) & (x_array < (L1 + L2) )] – L1)
Tramo3 = sp.lambdify(x, Cortante_3)(x_array[x_array >= (L1 + L2)] – L1 – L2)
#Graficar
dfc = np.concatenate((tramo1, tramo2, tramo3))
plt.plot(x_array, dfc)
plt.fill_between(x_array, dfc, alpha = 0,25)
#Comprobar cierre
plt.plot([0, 0], [0, float(Fuerza_Ay)], marker=”v”, color=”red”)
plt.plot([L_T, L_T], [0, -Py], marker=”v”, color=”red”)
#Grafica de ejes
plt.axvline(0, color=”gray”, linestyle=´--´)
plt.axhline(0, color=”gray”, linestyle=´--´)

In [ ]:
#MOMENTO
Tramo1_Momento = sp.lambdify(x, Momento_1)(x_array[x_array < L1])
Tramo2_Momento = sp.lambdify(x, Momento_2)(x_array[(x_array >= L1) & (x_array < (L1 + L2) )] -L1)
Tramo3_Momento = sp.lambdify(x, Momento_3)(x_array[x_array >= (L1 + L2)] – L1 – L2)
dmf = np.concatenate((Tramo1_Momento, Tramo2_Momento, Tramo3_Momento))
plt.plot(x_array, dmf)
plt.fill_between(x_array, dmf, alpha=0.25)

plt.plot([0,0], [0, -float(Momento_A)], marker=”v”, color=”red”)

#Grafica de ejes
plt.axvline(0,color=”gray”, linestyle=´--´)
plt.axhline(0,color=”gray”, linestyle=´--´)

In [ ]:
#Hallar cortante máximo

def Obtener_maximo(valores_diagrama):
    valor_max_positivo = np.amax(valores_diagrama)
    valor_max_negativo = np.amin(valores_diagrama)

    valor_maximo = valor_max_positivo
    if np.absolute(valor_max_negativo) > valor_max_positivo:
        valor_maximo = valor_max_negativo
    
    Posicion = x_array[np.where(valores_diagrama == valor_maximo)][0]
    return valor_maximo, Posicion

Cortante_maximo, Posicion_cortante_maximo = obtener_maximo(dfc)
print(f”El valor del cortante máximo es de {Cortante_maximo} kN”)
print(f”Se encuentra localizado a {Posicion_cortante_maximo} metros desde el nodo A”)   

In [ ]:
#Hallar momento máximo

Momento_maximo, posicion_momento_maximo = obtener_maximo(dmf)
print(f”El valor del momento máximo es de {momento_maximo} kN – m”)
print(f”Se encuentra localizado a {posicion_momento_maximo} metros desde el nodo A)